# One-layer MOSAIKS

## Load utils

In [1]:
import os
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F

/home/mila/s/sara.ebrahim-elkafrawy/.conda/envs/ffcv2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Define the model: two-layer CNN

In [12]:
# load the 2-layer mosaiks model
# mosaiks+kmeans: 
# two_layer_mosaiks_kmeans_7_no_whiten_zcaBias_0.001_bias_True_filters_100
# two_layer_mosaiks_kmeans_7_whiten_allGeo_bs32_25eachimage_zcaBias_0.001_bias_True_filters_100
# original mosaiks:
# two_layer_mosaiks_original_7_no_whiten_zcaBias_0.001_bias_True_filters_100
# two_layer_mosaiks_original_7_whiten_zcaBias_0.001_bias_True_filters_100

main_dir = "/home/mila/s/sara.ebrahim-elkafrawy/scratch/ecosystem_project/ckpts"
model_name = "two_layer_mosaiks_original_7_whiten_zcaBias_0.001_bias_True_filters_100.pt" 
num_filters_lyrs = [100, 64] #[512, 128] 
num_final_feats = 5184  #10368 (for 512 features)
conv_bias = True
num_species = 17037
# params to save the one-layer model
model_type = "one_layer_mosaiks_kmeans"
mode = "whiten"
kernel_size = 7
zca_bias = 0.001

In [13]:
# first trial: num_filters = [100, 64] with convolution bias and pool size of 256
# model = nn.Sequential(
#       nn.Conv2d(in_channels=3, out_channels=num_filters[0], kernel_size=kernel_size, padding='same', bias=True),
#       nn.LeakyReLU(),
#       nn.MaxPool2d(2, stride=2),

#       nn.Conv2d(in_channels=num_filters[0], out_channels=num_filters[1], kernel_size=kernel_size, padding='same', bias=True),
#       nn.LeakyReLU(),
#       nn.MaxPool2d(2, stride=2),
    
#       nn.AdaptiveAvgPool2d(9),
    
#       nn.Flatten(),
#       nn.Dropout(0.5),
#       nn.Linear(5184, 512), #50176
#       nn.ReLU(),
#       nn.Linear(512, num_species)
#       ) 

# second trial: num_filters = [512, 128] without conv bias and pool size of 2 
model = nn.Sequential(
      nn.Conv2d(in_channels=3, out_channels=num_filters_lyrs[0], kernel_size=kernel_size, padding='same', bias=conv_bias),
      nn.LeakyReLU(),
      nn.MaxPool2d(2, stride=2),

      nn.Conv2d(in_channels=num_filters_lyrs[0], out_channels=num_filters_lyrs[1], kernel_size=kernel_size, padding='same', bias=conv_bias),
      nn.LeakyReLU(),
      nn.MaxPool2d(2, stride=2),
    
      nn.AdaptiveAvgPool2d(9),
    
      nn.Flatten(),
      nn.Dropout(0.5),
      nn.Linear(num_final_feats, 512),
      nn.ReLU(),
      nn.Linear(512, num_species)
      )

model(torch.rand((1, 3, 224, 224))).shape
model_path = os.path.join(main_dir, model_name)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

# Get the first conv layer

In [17]:
one_conv_layer = model[0]
one_conv_layer.weight.requires_grad = False

In [18]:
type(one_conv_layer)

torch.nn.modules.conv.Conv2d

In [19]:
for param in one_conv_layer.parameters():
    print(param.requires_grad)

False
True


In [6]:
# model_path = os.path.join(main_dir, 'one_layer_'+'_'.join(model_name.split('_')[2:]))
# torch.save(one_conv_layer.state_dict(), model_path)
# print(f'Model saved at {model_path}')

Model saved at /home/mila/s/sara.ebrahim-elkafrawy/scratch/ecosystem_project/ckpts/one_layer_mosaiks_original_7_whiten_zcaBias_0.001_bias_False_filters_512.pt


# Construct the whole one layer model

In [7]:
pool_size = 2
pool_stride= 2
bias = 0.0
adaptive_pool_sz = 5

In [8]:
tmp_input =torch.rand((1, 3, 224, 224))

In [14]:
last_layer = nn.Linear(num_filters_lyrs[0] * adaptive_pool_sz * adaptive_pool_sz * 2, num_species)  
model = nn.Sequential(one_conv_layer, last_layer)

In [10]:
conv_output = one_conv_layer(tmp_input)

print(f'conv output shape: {conv_output.shape}')

conv output shape: torch.Size([1, 100, 224, 224])


In [11]:
x_pos = F.avg_pool2d(
    F.relu(conv_output - bias),
    [pool_size, pool_size],
    stride=[pool_stride, pool_stride],
    ceil_mode=True,
)

print(f'x_pos shape: {x_pos.shape}')

x_pos shape: torch.Size([1, 100, 112, 112])


In [12]:
x_neg = F.avg_pool2d(
    F.relu((-1 * conv_output) - bias),
    [pool_size, pool_size],
    stride=[pool_stride, pool_stride],
    ceil_mode=True,
)


In [13]:
avg_pool = nn.AdaptiveAvgPool2d(adaptive_pool_sz)
x_pos, x_neg = avg_pool(x_pos), avg_pool(x_neg)
print(x_pos.shape, x_neg.shape)

torch.Size([1, 100, 5, 5]) torch.Size([1, 100, 5, 5])


In [14]:
cat_vec = torch.cat((x_pos, x_neg), dim=1)
print(f"concatenated vector: {cat_vec.shape}")

concatenated vector: torch.Size([1, 200, 5, 5])


In [15]:
cat_vec = cat_vec.view(cat_vec.size(0), -1)
print(f"after reshape: {cat_vec.shape}")

after reshape: torch.Size([1, 5000])


In [18]:
final_output = last_layer(cat_vec)

In [19]:
final_output.shape

torch.Size([1, 17037])